In [1]:
# 在Jupyter Notebook中运行启发式生成代码
import argparse
import os
from src.pipeline.heuristic_generator import HeuristicGenerator
from src.util.llm_client.get_llm_client import get_llm_client

# 1. 定义参数（替代命令行输入）
# 在这里修改参数值，相当于命令行中的各种选项
problem = "tsp"  # 问题类型，如"tsp"、"cvrp"等
smoke_test = True  # 是否进行冒烟测试，相当于命令行的-m
llm_config_file = os.path.join("output", "llm_config", "azure_gpt_4o.json")  # LLM配置文件路径
source = "llm"  # 生成来源，可选"llm"、"paper"、"related_problem"
paper_path = None  # 当source为"paper"时，指定论文路径
related_problems = "all"  # 当source为"related_problem"时，指定相关问题
reference_data = None  # 参考数据集路径

# 2. 准备问题池（自动获取可用问题类型）
problem_pool = [p for p in os.listdir(os.path.join("src", "problems")) if p != "base"]
if problem not in problem_pool:
    raise ValueError(f"问题类型'{problem}'不存在，可用问题类型：{problem_pool}")

In [8]:
problem_pool

['cvrp', 'dposp', 'jssp', 'max_cut', 'mkp', 'tsp']

In [2]:
# 3. 初始化LLM客户端
prompt_dir = os.path.join("src", "problems", "base", "prompt")
output_dir = os.path.join("output", problem, "generate_heuristic")

In [3]:
prompt_dir

'src\\problems\\base\\prompt'

In [4]:
output_dir

'output\\tsp\\generate_heuristic'

In [5]:
def parse_arguments():
    problem_pool = [problem for problem in os.listdir(os.path.join("src", "problems")) if problem != "base"]

    parser = argparse.ArgumentParser(description="Generate heuristic")
    parser.add_argument("-p", "--problem", choices=problem_pool, required=True, help="Specifies the type of combinatorial optimization problem.")
    parser.add_argument("-m", "--smoke_test", action='store_true', help="Optional flag to conduct a preliminary smoke test.")
    parser.add_argument("-l", "--llm_config_file", type=str, default=os.path.join("output", "llm_config", "azure_gpt_4o.json"), help="Path to the language model configuration file. Default is azure_gpt_4o.json.")
    parser.add_argument("-s", "--source", choices=["llm", "paper", "related_problem"], default="llm", help="Source for generating heuristics: 'llm' for automated model generation, 'paper' for literature-based methods, 'related_problem' for adaptations from similar problems.")
    parser.add_argument("-pp", "--paper_path", type=str, help="Specify the path to a LaTeX paper file or directory containing heuristic descriptions.")
    parser.add_argument("-r", "--related_problems", type=str, default="all", help="Comma-separated list of related problem domains to draw heuristic insights from.")
    parser.add_argument("-d", "--reference_data", type=str, default=None, help="Optional path to reference datasets, used when generating heuristics tailored to specific data distributions.")

    return parser.parse_args()

In [6]:
llm_api_endpoint = "https://api.deepseek.com", # 设置LLM（大语言模型）的API端点（需替换为实际地址）
llm_api_key = "sk-1795d95c9b4c4f3e9c54c80c1f5e5f31",   # 设置LLM的API密钥（需替换为实际密钥）
llm_model = "deepseek-chat", # 设置使用的LLM模型名称（需替换为实际模型名）

In [7]:
# 在Jupyter Notebook中运行启发式生成代码
import argparse
import os
from src.pipeline.heuristic_generator import HeuristicGenerator
from src.util.llm_client.get_llm_client import get_llm_client

# 1. 定义参数（替代命令行输入）
# 在这里修改参数值，相当于命令行中的各种选项
problem = "tsp"  # 问题类型，如"tsp"、"cvrp"等
smoke_test = True  # 是否进行冒烟测试，相当于命令行的-m
llm_config_file = os.path.join("output", "llm_config", "azure_gpt_4o.json")  # LLM配置文件路径
source = "llm"  # 生成来源，可选"llm"、"paper"、"related_problem"
paper_path = None  # 当source为"paper"时，指定论文路径
related_problems = "all"  # 当source为"related_problem"时，指定相关问题
reference_data = None  # 参考数据集路径

# 2. 准备问题池（自动获取可用问题类型）
problem_pool = [p for p in os.listdir(os.path.join("src", "problems")) if p != "base"]
if problem not in problem_pool:
    raise ValueError(f"问题类型'{problem}'不存在，可用问题类型：{problem_pool}")

# 3. 初始化LLM客户端
prompt_dir = os.path.join("src", "problems", "base", "prompt")
output_dir = os.path.join("output", problem, "generate_heuristic")

# 创建输出目录（如果不存在）
os.makedirs(output_dir, exist_ok=True)

# 获取LLM客户端
llm_client = get_llm_client(llm_config_file, prompt_dir, output_dir)

# 4. 初始化启发式生成器
heuristic_generator = HeuristicGenerator(llm_client=llm_client, problem=problem)

# 5. 根据来源生成启发式算法
if source == "llm":
    # 从LLM生成
    result = heuristic_generator.generate_from_llm(
        reference_data=reference_data,
        smoke_test=smoke_test
    )
elif source == "paper":
    # 从论文生成（需指定paper_path）
    if not paper_path:
        raise ValueError("当source为'paper'时，必须指定paper_path")
    result = heuristic_generator.generate_from_paper(
        paper_path=paper_path,
        reference_data=reference_data,
        smoke_test=smoke_test
    )
elif source == "related_problem":
    # 从相关问题迁移
    if related_problems == "all":
        related_problems_list = [p for p in problem_pool if p not in ["base", problem]]
    else:
        related_problems_list = related_problems.split(",")
    
    result = heuristic_generator.generate_from_reference(
        related_problems=related_problems_list,
        reference_data=reference_data,
        smoke_test=smoke_test
    )
else:
    raise ValueError(f"未知的生成来源：{source}，可用来源：['llm', 'paper', 'related_problem']")

print(f"启发式算法生成完成，结果保存至：{output_dir}")


Chat dumped to output\tsp\generate_heuristic\background.txt
Try to chat 1 time: Expecting value: line 1 column 1 (char 0)
Try to chat 2 time: Expecting value: line 1 column 1 (char 0)
Try to chat 3 time: Expecting value: line 1 column 1 (char 0)
Try to chat 4 time: Expecting value: line 1 column 1 (char 0)
Try to chat 5 time: Expecting value: line 1 column 1 (char 0)
Try to chat 6 time: Expecting value: line 1 column 1 (char 0)
Try to chat 7 time: Expecting value: line 1 column 1 (char 0)
Try to chat 8 time: Expecting value: line 1 column 1 (char 0)
Try to chat 9 time: Expecting value: line 1 column 1 (char 0)
Try to chat 10 time: Expecting value: line 1 column 1 (char 0)
Try to chat 11 time: Expecting value: line 1 column 1 (char 0)
Try to chat 12 time: Expecting value: line 1 column 1 (char 0)
Try to chat 13 time: Expecting value: line 1 column 1 (char 0)
Try to chat 14 time: Expecting value: line 1 column 1 (char 0)
Try to chat 15 time: Expecting value: line 1 column 1 (char 0)
Try 

TypeError: string indices must be integers, not 'str'